In [2]:
!pip install pymupdf faiss-cpu sentence-transformers transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [3]:
import fitz
import faiss
import numpy as np

In [4]:
def read_text(doc):
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [9]:
doc=fitz.open(r"/content/Brene-brown-book1.pdf")
raw_text = read_text(doc)

raw_text

'Advance praise for Daring Greatly\n“A wonderful book: urgent, essential, and fun to read. I couldn’t put it down, and\nit continues to resonate with me.”\n—Seth Godin, New York Times bestselling author of Linchpin\n“The brilliantly insightful Brené Brown draws upon extensive research and\npersonal experience to explore the paradoxes of courage: We become strong by\nembracing vulnerability, we dare more greatly when we acknowledge our fear. I\ncan’t stop thinking about this book.”\n—Gretchen Rubin, New York Times bestselling author of\nThe Happiness Project and Happiness at Home\n“In Daring Greatly, Brené Brown refers to herself as both a mapmaker and a\ntraveler. In my book, that makes her a guide. And I believe the world needs more\nguides like her who are showing us a wiser way to our inner world. If you’d like\nto set your course on being more courageous and connected, engaged and\nresilient, leave the GPS at home. Daring Greatly is all the navigation you’ll\nneed.”\n—Maria Shriver

In [10]:
def split_into_chunks(text, chunk_size=500):
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunk = text[i:i + chunk_size]
        chunks.append(chunk)
    return chunks

In [11]:
from sentence_transformers import SentenceTransformer


In [12]:
model=SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
chunks = split_into_chunks(raw_text)

In [14]:
embeddings=model.encode(chunks)

In [17]:

#  Convert list of embeddings into a NumPy array (2D)
embedding_array = np.array(embeddings).astype("float32")

#  Just to be safe: reshape if it's only 1D
if len(embedding_array.shape) == 1:
    embedding_array = embedding_array.reshape(1, -1)

#  Create a FAISS index
dimension = embedding_array.shape[1]  # size of each vector (should be 384)
index = faiss.IndexFlatL2(dimension)

# Add your embeddings to it
index.add(embedding_array)


print("Chunks stored in FAISS:", index.ntotal)


Chunks stored in FAISS: 883


In [18]:
# print(len(chunks))
print(chunks[0] if len(chunks) > 0 else "No chunks found")


Advance praise for Daring Greatly
“A wonderful book: urgent, essential, and fun to read. I couldn’t put it down, and
it continues to resonate with me.”
—Seth Godin, New York Times bestselling author of Linchpin
“The brilliantly insightful Brené Brown draws upon extensive research and
personal experience to explore the paradoxes of courage: We become strong by
embracing vulnerability, we dare more greatly when we acknowledge our fear. I
can’t stop thinking about this book.”
—Gretchen Rubin, New Y


In [19]:
print(len(embeddings))
print(type(embeddings[0]))


883
<class 'numpy.ndarray'>


In [20]:
print(index.ntotal)

883


In [21]:
def get_context(question, top_k=1):
    question_embedding = model.encode([question]).astype("float32")
    print("Shape of question embedding:", question_embedding.shape)

    distances, indices = index.search(question_embedding, top_k)
    print("Indices returned:", indices)

    valid_indices = [i for i in indices[0] if i != -1]

    if not valid_indices:
        return "No relevant context found."

    results = [chunks[i] for i in valid_indices]
    return "\n\n".join(results)


In [22]:
from transformers import pipeline

qa_model = pipeline("text-generation", model="gpt2")

def chatbot(query):
    context = get_context(query)
    prompt = f"Answer based on this:\n{context}\n\nQ: {query}\nA:"
    response = qa_model(prompt, max_length=150, do_sample=True)[0]['generated_text']
    return response


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [23]:
print("🤖 Hello! I'm your PDF Assistant. Ask me anything from the document.")
print("Type 'exit' or 'quit' anytime to end the chat.\n")

while True:
    user_input = input("👤 You: ")
    if user_input.strip().lower() in ["exit", "quit"]:
        print("🤖 Bot: Goodbye! 👋")
        break

    response = chatbot(user_input)

    if response.strip() == "":
        print("🤖 Bot: Hmm... I couldn't find an answer in the document. Try rephrasing?")
    else:
        print(f"🤖 Bot: {response.strip()}\n")

🤖 Hello! I'm your PDF Assistant. Ask me anything from the document.
Type 'exit' or 'quit' anytime to end the chat.

👤 You: whats the name of book


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Shape of question embedding: (1, 384)
Indices returned: [[5]]
🤖 Bot: Answer based on this:
es as she speaks, with wisdom, wit,
candor, and a deep sense of humanity. If you’re a student, teacher, parent,
employer, employee, or just alive and wanting to live more fully, you should
read this book. I double dare you.”
—Sir Ken Robinson, New York Times bestselling author of
Out of Our Minds and The Element:
How Finding Your Passion Changes Everything
“Here’s the essence of this book: Vulnerability is courage in you but inadequacy
in me. Brené’s book, weaving together research and Texan ane

Q: whats the name of book
A:: The Book of Knowledge, the greatest
theory of all time. It's the greatest book on
the subject. It's the most
knowing book ever written.“““““““
“““““““““““““““““““““““““““““““““
“““““““““““““““““““““““““““““““““““““““““““““““““““““““““““““““““““““

👤 You: author name


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Shape of question embedding: (1, 384)
Indices returned: [[2]]
🤖 Bot: Answer based on this:
bestselling author of
Just Who Will You Be?
“Daring Greatly is an important book—a timely warning about the danger of
pursuing certainty and control above all. Brené Brown offers all of us a valuable
guide to the real reward of vulnerability: greater courage.”
—Daniel H. Pink, New York Times bestselling author of
Drive and A Whole New Mind
“What I find remarkable about this book is the unique combination of solid
research and kitchen-table storytelling. Brené becomes such a real person in the
bo

Q: author name
A: Daniel H. Pink is the author of
The Way We Think. His books are accessible and thoughtful—he is a

true innovator.
—Michael S. Ettlinger, New York Times bestselling author of

The Way We Think.
Q: what would you like me to write about
“How to Start a Business?“
“What is your experience with business?“““

👤 You: the last one


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Shape of question embedding: (1, 384)
Indices returned: [[723]]
🤖 Bot: Answer based on this:
 captain says, ‘I’ll take Suzie, John, Pete, Robin,
and Jake.’ The second captain says, ‘I’ll take Andrew, Steve, Katie, and Sue, and
we can split the others.’ Every single day I’m one of the others. I never get to be
named.”
My heart sank. She was sitting on the edge of her bed with her head in her
hands. I was so concerned when I followed her into her room that I hadn’t even
flipped on the light. I couldn’t stand the vulnerability of seeing her sitting in the
dark crying, so I walked over to t

Q: the last one
A: I sat with her, and I watched her in pain. In
the dark, in the dark, in the dark, in the dark, in the dark. †
My heart sank. I was so concerned when I followed her into her room that I hadn’t evenflipped on the light. I couldn’t stand the vulnerability of seeing her sitting in thedark crying, so I sat with her, and I watched her in pain. Inthe dark, in the dark, in the dark, in the d

In [33]:
import gradio as gr
import fitz
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Load model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Globals
chunks = []
index = None

# Chatbot logic
def chatbot_interface(query):
    if query.strip().lower() in ["exit", "quit"]:
        return "👋 Thanks for chatting! See you soon."

    response = get_response(query)
    return response.strip() if response.strip() else "🤖 Sorry, I couldn't find an answer. Try rephrasing?"

# Search in PDF chunks
def get_response(query):
    global chunks, index
    q_embedding = model.encode([query]).astype("float32")
    distances, indices = index.search(q_embedding, k=1)
    context = chunks[indices[0][0]]
    return f"📖 Context:\n{context}\n\n💬 Answer (dummy): GPT-2 would answer here."

# ✅ Fix is applied here: using `fitz.open(file)` instead of `.read()`
def process_pdf(file):
    try:
        global chunks, index
        doc = fitz.open(file)  # ✅ Fix: pass file path directly

        text = ""
        for page in doc:
            text += page.get_text()
        doc.close()

        chunks = [text[i:i + 500] for i in range(0, len(text), 500)]
        embeddings = model.encode(chunks).astype("float32")
        index = faiss.IndexFlatL2(embeddings.shape[1])
        index.add(embeddings)

        return "✅ PDF uploaded and processed successfully!"
    except Exception as e:
        import traceback
        print(traceback.format_exc())
        return f"❌ Error: {str(e)}"

# CSS (replace with your actual CSS if needed)
css = """
body {
    background: linear-gradient(to right, #e0eafc, #cfdef3);
    animation: fadeIn 1s ease-in;
}
h1, h2, p {
    font-family: 'Segoe UI', sans-serif !important;
    color: #2c3e50;
    text-align: center;
}
.gradio-container {
    font-family: 'Segoe UI', sans-serif;
    padding: 20px;
}
textarea {
    font-size: 16px !important;
    border-radius: 12px !important;
    border: 1px solid #dcdde1 !important;
    box-shadow: 0 0 5px rgba(0,0,0,0.1) !important;
    transition: all 0.3s ease-in-out;
}
textarea:focus {
    border-color: #3498db !important;
    box-shadow: 0 0 8px #3498db50 !important;
}
button {
    background-color: #3498db !important;
    color: white !important;
    font-size: 16px !important;
    border-radius: 10px !important;
    padding: 10px 20px !important;
    transition: background-color 0.3s ease;
}
button:hover {
    background-color: #2980b9 !important;
}
@keyframes fadeIn {
    0% { opacity: 0; transform: translateY(10px); }
    100% { opacity: 1; transform: translateY(0); }
}
"""

# Gradio interface
with gr.Blocks(css=css) as demo:
    gr.Markdown("## 📚 AI PDF Chatbot")
    gr.Markdown("""
    <div style='text-align: center; font-size: 16px; margin-top: 10px;'>
        🤖 Ask questions based on your PDF.<br>
        Uses <strong>FAISS</strong> for context & <strong>GPT-2</strong> to generate answers.<br>
        Upload a PDF first, then start chatting!
    </div>
    """)

    with gr.Row():
        pdf_input = gr.File(label="📄 Upload your PDF", file_types=[".pdf"])
        upload_btn = gr.Button("📤 Process PDF")

    upload_status = gr.Textbox(label="Status", interactive=False)
    upload_btn.click(fn=process_pdf, inputs=pdf_input, outputs=upload_status)

    with gr.Row():
        question = gr.Textbox(lines=3, label="📌 Your Question", placeholder="Ask something from the uploaded PDF...")
        answer = gr.Textbox(label="🤖 Bot's Answer")
    ask_btn = gr.Button("💬 Ask")  # <-- Added Ask button
    ask_btn.click(fn=chatbot_interface, inputs=question, outputs=answer)


demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://864a444f82238f40c4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2191, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1702, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7863 <> https://864a444f82238f40c4.gradio.live
